## Inference:

In [131]:
%reset -f
import os, json 
import pandas as pd
dr = os.listdir('/home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/')
dr.sort()
dr

['.ipynb_checkpoints',
 'LSTM-AE_Results_SqL2-ftur242-f0',
 'LSTM-AE_Results_SqL2-ftur242-f14',
 'LSTM-AE_Results_SqL2-ftur242-f19',
 'LSTM-AE_Results_SqL2-ftur242-f4',
 'LSTM-AE_Results_SqL2-ftur242-f9',
 'LSTM-AE_Results_SqL2-ftur242-f99']

In [122]:
# 'LSTM-AE_Results_SqL2-ftur242-f0' OK
# 'LSTM-AE_Results_SqL2-ftur242-f14' OK
# 'LSTM-AE_Results_SqL2-ftur242-f19' OK
# 'LSTM-AE_Results_SqL2-ftur242-f4' OK
# 'LSTM-AE_Results_SqL2-ftur242-f9' OK
# 'LSTM-AE_Results_SqL2-ftur242-f99'

SyntaxError: invalid syntax (<ipython-input-122-4faebf84fac7>, line 1)

In [136]:
dir_path = '/home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/LSTM-AE_Results_SqL2-ftur242-f99/'
fl = os.listdir(dir_path)
fl.sort()
fl

['.ipynb_checkpoints',
 'Inference_Results-2.csv',
 'Inference_Results.csv',
 'model_epoch_10.pt',
 'model_epoch_100.pt',
 'model_epoch_20.pt',
 'model_epoch_30.pt',
 'model_epoch_40.pt',
 'model_epoch_50.pt',
 'model_epoch_60.pt',
 'model_epoch_70.pt',
 'model_epoch_80.pt',
 'model_epoch_90.pt',
 'output.txt',
 'scaler.pkl']

In [133]:
output = dir_path+'output.txt'
scaler_path = dir_path+'scaler.pkl'
model_path = dir_path+'model_epoch_100.pt'
Inference_Results_file = dir_path+'Inference_Results.csv'

with open(output) as text:
    txt = text.read()
restults_complete = json.loads(txt.split('#+#')[1])
print(restults_complete.keys())

print(f'Inference_Results_file: {Inference_Results_file}')
print(f'scaler_path: {scaler_path}')
print(f'output: {output}')
print(f'model_path: {model_path}')

inf_res = pd.read_csv(Inference_Results_file)

dict_keys(['Test_Size', 'Normal_Sum_Flags_Threshold', 'Batch_Size_Train', 'Batch_Size_Test', 'Shuffle', 'Rack', 'Number_of_Epochs', 'Learning_Rate', 'Device', 'Criterion', 'Optimizer', 'Abnormal_data_index', 'Normal_data_train_index', 'Normal_data_test_index', 'Number_featuers', 'columns', 'Time_Window', 'Epochs_Results'])
Inference_Results_file: /home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/LSTM-AE_Results_SqL2-ftur242-f99/Inference_Results.csv
scaler_path: /home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/LSTM-AE_Results_SqL2-ftur242-f99/scaler.pkl
output: /home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/LSTM-AE_Results_SqL2-ftur242-f99/output.txt
model_path: /home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM/LSTM-AE_Results_SqL2-ftur242-f99/model_epoch_100.pt


In [134]:
import os, sys, imp
sys.path.append('/home/seyedkazemi/codes/mskhelper/')
import pandas as pd, numpy as np
import datetime, pytz
import data_provider, mohsenutils, m100_preprocessing_helper, flag_helper
import imp 
from  matplotlib import pyplot as plt
import torch 
import torch.nn as nn
import torch.functional as f
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import autoencoder_helper
from pickle import load
import json 

imp.reload(data_provider)
imp.reload(mohsenutils)
imp.reload(m100_preprocessing_helper)
imp.reload(flag_helper)
imp.reload(autoencoder_helper)

import random
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True








# Time_Window = datetime.timedelta(hours=6)
Time_Window = datetime.timedelta(minutes=20)

with open(output) as text:
    txt = text.read()
restults_complete = json.loads(txt.split('#+#')[1])
print(restults_complete.keys())



dataframe = autoencoder_helper.flags_data(rack=restults_complete['Rack'], 
                                          ambient_dt=True,
                                          pcie_dt=True,
                                          fsnd_dt=True,
                                          psnd_dt=True,
                                          gpu0_dt=True,
                                          gpu1_dt=True,
                                          gpu3_dt=True,
                                          gpu4_dt=True,
                                          cpu0_dt=True,
                                          cpu1_dt=True,
                                          Tot_ict_dt=True,
                                          Tot_qpompe_dt=True, 
                                          RDHX_dt=True,
                                          Tot_chiller_dt=True,
                                          Supply_Air_Temperature_dt=True,
                                          Return_Air_Temperature_dt=True, 
                                          Free_Cooling_Valve_Open_Position_dt=True,
                                          Tot_cdz_dt=True, 
                                          FScrac_dt=True,
                                          Free_Cooling_Status_dt=True,
                                          Compressor_Utilization_dt=True,
                                          temp_dt=True)


print(f'Sum of null {dataframe.isnull().sum()[dataframe.isnull().sum()>0]}')
print(f'Sum of null {dataframe.T.isnull().sum()[dataframe.T.isnull().sum()>0]}')
dataframe = dataframe.interpolate()
print(f'Sum of null after interpolation: {dataframe.isnull().sum()[dataframe.isnull().sum()>0]}')
display(dataframe)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path).to(device)
display(model)
criterion = torch.nn.L1Loss(reduction='sum').to(device)


# load the scaler
scaler = load(open(scaler_path, 'rb'))
dataframe_scl = pd.DataFrame(scaler.transform(dataframe.drop('sum_flags', axis=1)), columns=dataframe.drop('sum_flags', axis=1).columns)
dataframe_scl.index = dataframe.index

all_dataset = autoencoder_helper.dataset_LSTM(dataframe=dataframe_scl, Time_Window=Time_Window, Index_lst=dataframe_scl.iloc[37:,:].index)

inference_results = pd.DataFrame(columns=['Error'])
for X in DataLoader(all_dataset):
    model.eval()
    dt, cols, idx = X
    x = torch.transpose(dt,1,2).to(device).float()
    pred = model(x)
    loss = criterion(pred, x)
    inference_results.loc[idx[-1][0],'Error']=loss.item()
    
    
inference_results.index.name = 'timestamp'
inference_results.index = pd.to_datetime(inference_results.index)
inference_results['Label'] =''
inference_results.loc[restults_complete['Normal_data_train_index'],'Label'] = 'train'
inference_results.loc[restults_complete['Normal_data_test_index'],'Label'] = 'test'
inference_results.loc[restults_complete['Abnormal_data_index'],'Label'] = 'abnormal'
inference_results = mohsenutils.mrg(None,'inner',dataframe[['sum_flags']], inference_results)
inference_results.to_csv(dir_path+'Inference_Results-2.csv')

dict_keys(['Test_Size', 'Normal_Sum_Flags_Threshold', 'Batch_Size_Train', 'Batch_Size_Test', 'Shuffle', 'Rack', 'Number_of_Epochs', 'Learning_Rate', 'Device', 'Criterion', 'Optimizer', 'Abnormal_data_index', 'Normal_data_train_index', 'Normal_data_test_index', 'Number_featuers', 'columns', 'Time_Window', 'Epochs_Results'])
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:50:00
2021-04-08 17:20:00 2021-05-21 23:5

,sum_flags,r205n01_ambient,r205n02_ambient,r205n03_ambient,r205n04_ambient,r205n05_ambient,r205n06_ambient,r205n07_ambient,r205n08_ambient,r205n09_ambient,...,cdz2_FCS,cdz3_FCS,cdz4_FCS,cdz1_CU,cdz2_CU,cdz3_CU,cdz4_CU,cdz5_CU,cdz6_CU,current
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-08 17:20:00,7,20.253333,19.373333,21.166667,22.246667,23.600000,23.640000,23.460000,23.800000,24.200000,...,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,25.0,0.000000,16.00
2021-04-08 17:30:00,10,20.540000,19.753333,21.600000,22.773333,23.893333,23.786667,23.600000,23.800000,24.340000,...,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,25.0,0.000000,16.00
2021-04-08 17:40:00,10,20.426667,19.640000,21.546667,22.713333,23.846667,23.773333,23.600000,23.800000,24.333333,...,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,25.0,0.000000,16.00
2021-04-08 17:50:00,4,20.540000,19.760000,21.546667,22.600000,23.773333,23.726667,23.586667,23.800000,24.273333,...,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,25.0,0.000000,16.00
2021-04-08 18:00:00,3,20.546667,19.760000,21.666667,22.786667,23.906667,23.780000,23.586667,23.813333,24.346667,...,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,25.0,0.000000,16.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-21 23:10:00,9,18.986667,17.840000,19.100000,18.506667,20.446667,22.093333,23.060000,23.986667,24.600000,...,1.0,1.0,1.0,48.750000,0.0,0.0,0.0,50.0,14.166667,16.68
2021-05-21 23:20:00,8,18.833333,17.446667,18.686667,17.933333,20.040000,21.826667,22.893333,23.900000,24.560000,...,1.0,1.0,1.0,69.166667,0.0,0.0,0.0,50.0,25.000000,16.64
2021-05-21 23:30:00,9,18.706667,17.413333,18.706667,17.920000,19.880000,21.666667,22.780000,23.886667,24.566667,...,1.0,1.0,1.0,60.416667,0.0,0.0,0.0,50.0,25.000000,16.64


RecurrentAutoencoder(
  (encoder): Encoder(
    (rnn1): LSTM(242, 128, batch_first=True)
    (rnn2): LSTM(128, 64, batch_first=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(64, 64, batch_first=True)
    (rnn2): LSTM(64, 128, batch_first=True)
    (output_layer): Linear(in_features=128, out_features=242, bias=True)
  )
)

In [135]:
import pandas as pd

d1 = pd.read_csv(dir_path + 'Inference_Results.csv', index_col='timestamp')
d2 = pd.read_csv(dir_path + 'Inference_Results-2.csv', index_col='timestamp')
print(f'\td1 Null:\n{d1.isnull().sum(axis=0)} \n\n\td2 Null:\n{d2.isnull().sum(axis=0)}')
print('\n\td1-d2:')
print(np.abs((d1.drop('Label', axis=1) - d2.drop('Label', axis=1))).sum())

display(d1, d2)


	d1 Null:
sum_flags    0
Error        0
Label        1
dtype: int64 

	d2 Null:
sum_flags    0
Error        0
Label        1
dtype: int64

	d1-d2:
sum_flags    0.0
Error        0.0
dtype: float64


,sum_flags,Error,Label
timestamp,,,
2021-04-08 23:30:00,7,47.900307,NaN
2021-04-08 23:40:00,6,42.815235,train
2021-04-08 23:50:00,6,42.250347,train
2021-04-09 00:00:00,15,41.229950,train
2021-04-09 00:10:00,25,47.190620,train
...,...,...,...
2021-05-21 23:10:00,9,43.229919,train
2021-05-21 23:20:00,8,45.768127,train
2021-05-21 23:30:00,9,44.711628,train


,sum_flags,Error,Label
timestamp,,,
2021-04-08 23:30:00,7,47.900307,NaN
2021-04-08 23:40:00,6,42.815235,train
2021-04-08 23:50:00,6,42.250347,train
2021-04-09 00:00:00,15,41.229950,train
2021-04-09 00:10:00,25,47.190620,train
...,...,...,...
2021-05-21 23:10:00,9,43.229919,train
2021-05-21 23:20:00,8,45.768127,train
2021-05-21 23:30:00,9,44.711628,train


In [ ]:
import glob
datename = '20217110522'
output = glob.glob(dir_path+'output*.txt')[0]

with open(output) as text:
    txt = text.read()
restults_complete = json.loads(txt.split('#+#')[1])
print(restults_complete.keys())

scaler_path = glob.glob(dir_path+'scaler*.pkl')[0]
model_path = glob.glob(dir_path+'model_epoch_100_*.pt')[0]
Inference_Results_file = glob.glob(dir_path+'Inference_Results*.csv')[0]


print(f'datename: {datename}')
print(f'Inference_Results_file: {Inference_Results_file}')
print(f'scaler_path: {scaler_path}')
print(f'output: {output}')
print(f'model_path: {model_path}')



pd.read_csv(Inference_Results_file)

In [ ]:
import pandas as pd

d1 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM-AE_Results-242-f0/Inference_Results_2021630114412.csv', index_col='timestamp')
d2 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100-Cooked/AE/LSTM-AE_Results-242-f0/Inference_Results_20216301144122.csv', index_col='timestamp')
display(d1, d2)

In [ ]:
(d1.drop('Label', axis=1) - d2.drop('Label', axis=1)).sum()

In [ ]:
import os, sys, imp
sys.path.append('/home/seyedkazemi/codes/mskhelper/')
import pandas as pd, numpy as np
import datetime, pytz
import data_provider, mohsenutils, m100_preprocessing_helper, flag_helper
import imp 
from  matplotlib import pyplot as plt
import torch 
import torch.nn as nn
import torch.functional as f
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import autoencoder_helper
from pickle import load
import json 

imp.reload(data_provider)
imp.reload(mohsenutils)
imp.reload(m100_preprocessing_helper)
imp.reload(flag_helper)
imp.reload(autoencoder_helper)

import random
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True








# Time_Window = datetime.timedelta(hours=6)
Time_Window = datetime.timedelta(minutes=20)

with open(output) as text:
    txt = text.read()
restults_complete = json.loads(txt.split('#+#')[1])
print(restults_complete.keys())



dataframe = autoencoder_helper.flags_data(rack=restults_complete['Rack'], 
                                          ambient_dt=True,
                                          pcie_dt=True,
                                          fsnd_dt=True,
                                          psnd_dt=True,
                                          gpu0_dt=True,
                                          gpu1_dt=True,
                                          gpu3_dt=True,
                                          gpu4_dt=True,
                                          cpu0_dt=True,
                                          cpu1_dt=True,
                                          Tot_ict_dt=True,
                                          Tot_qpompe_dt=True, 
                                          RDHX_dt=True,
                                          Tot_chiller_dt=True,
                                          Supply_Air_Temperature_dt=True,
                                          Return_Air_Temperature_dt=True, 
                                          Free_Cooling_Valve_Open_Position_dt=True,
                                          Tot_cdz_dt=True, 
                                          FScrac_dt=True,
                                          Free_Cooling_Status_dt=True,
                                          Compressor_Utilization_dt=True,
                                          temp_dt=True)


display(dataframe)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path).to(device)
display(model)
criterion = torch.nn.L1Loss(reduction='sum').to(device)


# load the scaler
scaler = load(open(scaler_path, 'rb'))
dataframe_scl = pd.DataFrame(scaler.transform(dataframe.drop('sum_flags', axis=1)), columns=dataframe.drop('sum_flags', axis=1).columns)


dataframe_scl.index = dataframe.index
all_dataset = autoencoder_helper.dataset_LSTM(dataframe=dataframe_scl, Time_Window=Time_Window, Index_lst=dataframe_scl.iloc[37:,:].index)

inference_results = pd.DataFrame(columns=['Error'])

for X in DataLoader(all_dataset):
    model.eval()
    dt, cols, idx = X
    x = torch.transpose(dt,1,2).to(device).float()
    pred = model(x)
    loss = criterion(pred, x)
    inference_results.loc[idx[-1][0],'Error']=loss.item()
    
    
    
inference_results.index.name = 'timestamp'
inference_results.index = pd.to_datetime(inference_results.index)
inference_results['Label'] =''
inference_results.loc[restults_complete['Normal_data_train_index'],'Label'] = 'train'
inference_results.loc[restults_complete['Normal_data_test_index'],'Label'] = 'test'
inference_results.loc[restults_complete['Abnormal_data_index'],'Label'] = 'abnormal'
inference_results = mohsenutils.mrg(None,'inner',dataframe[['sum_flags']], inference_results)
inference_results.to_csv(dir_path+'Inference_Results_'+str(datename)+'.csv')
                         
